<a href="https://colab.research.google.com/github/dqrtec/IA/blob/master/formiga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import random as rdn

In [39]:
class formiga:
    caminho = None
    matrizRainhas = None
    matrizFeromonio = None
    custo = None
    
    def __init__(self, rainhas):
        self.caminho = self.caminhoInicial(rainhas)
        self.geraMatrizRainhas(rainhas)
        self.criarMatrizFeromonio(rainhas)
        self.custo = rainhas
    
    def caminhoInicial(self, n):
        a = [i for i in range(n)] 
        rdn.shuffle(a)
        return a
    
    def geraMatrizRainhas(self, n):
        self.matrizRainhas = [[0 for i in range(n)] for j in range(n)]

        for l, c in enumerate(self.caminho):
            self.matrizRainhas[l][c] = 1
    
    def calcularCusto(self):
        custoLinha = 0
        dicionario = {}
        for i in self.caminho:
            if i in dicionario:
                custoLinha += 1
                dicionario[i] = dicionario[i] + 1
            else:
                dicionario[i] = 1

        somaDiagonal = 0 
        
        for i in range(len(self.caminho)):
            for j in range(i+1, len(self.caminho)):

                if self.caminho[i]+(j-i) < len(self.caminho):
                    if self.caminho[i+(j-i)] == self.caminho[i]+(j-i):
                        somaDiagonal += 1
                if self.caminho[i]-(j-i) >= 0:
                    if self.caminho[i+(j-i)] == self.caminho[i]-(j-i):
                        somaDiagonal += 1

            for j in range(0,i):
                if self.caminho[i]+(j+1) < len(self.caminho):
                    if self.caminho[i-(j+1)] == self.caminho[i]+(j+1):
                        somaDiagonal += 1
                if self.caminho[i]-(j+1) >= 0:
                    if self.caminho[i-(j+1)] == self.caminho[i]-(j+1):
                        somaDiagonal += 1
                        
        self.custo = int(somaDiagonal / 2) + custoLinha
    
    def criarMatrizFeromonio(self, N):
        self.matrizFeromonio = [[1 for i in range(N)] for j in range(N)]
    
    def calcularMatrizFeromonio(self):
        if (self.custo == 0):
            return self.caminho

        feromonio = round(1 / self.custo, 3)

        for l, c in enumerate(self.caminho):
            self.matrizFeromonio[l][c] += feromonio
                
                
    def calcMatrizProb(self):
        matrizProb = []
        somas = []
        
        for line in self.matrizFeromonio:
            somas.append(0)
            
        for line in self.matrizFeromonio:    
            for index, col in enumerate(line):
                somas[index] += col

        for indexLine, line in enumerate(self.matrizFeromonio):
            matrizProb.append([])
            
            for indexCol, col in enumerate(line):
                matrizProb[indexLine].append(round(col / somas[indexCol], 3))
                      
        return matrizProb
        
    def novoCaminhoFormiga(self, matrizProb):
        novoCaminho = [0 for n in matrizProb]
        i = 0
        for line in matrizProb:
            roleta = [0 for n in line]
            pesos = [n for n in line]
            somaPeso = sum(pesos)
            sorteio = rdn.random()
            
            prob = [x*100/somaPeso for x in line]
            
            roleta[0] = (0, prob[0])
            for x in range(1, len(prob)):
                roleta[x] = (roleta[x-1], roleta[x-1] + prob[x])

            novoCaminho[i] = escolherPosicaoArbitraria(roleta, sorteio)
            i += 1

        return novoCaminho
    
    def escolherPosicaoArbitraria(self, roleta, sorteio):
        posicao = 0
        for minimo, maximo in roleta:
            if sorteio >= minimo and sorteio < maximo
                return posicao
        posicao += 
    
    def debug(self, matrizProb):
        print('Caminho', self.caminho)
        
        print('Rainhas')
        self.printMatriz(self.matrizRainhas)
        
        print('Feromonio')
        self.printMatriz(self.matrizFeromonio)
        
        print('Probabilidade')
        self.printMatriz(matrizProb)
        
    def printMatriz(self, matriz):
        for linha in matriz:
            print(linha)
        print()

In [40]:
Antony = formiga(5)

while (Antony.custo > 0):
    Antony.calcularMatrizFeromonio()
    
    matrizProb = Antony.calcMatrizProb()
    Antony.debug(matrizProb)
    
    Antony.caminho = Antony.novoCaminhoFormiga(matrizProb)
    

Caminho [1, 0, 3, 2, 4]
Rainhas
[0, 1, 0, 0, 0]
[1, 0, 0, 0, 0]
[0, 0, 0, 1, 0]
[0, 0, 1, 0, 0]
[0, 0, 0, 0, 1]

Feromonio
[1, 1.2, 1, 1, 1]
[1.2, 1, 1, 1, 1]
[1, 1, 1, 1.2, 1]
[1, 1, 1.2, 1, 1]
[1, 1, 1, 1, 1.2]

Probabilidade
[0.192, 0.231, 0.192, 0.192, 0.192]
[0.231, 0.192, 0.192, 0.192, 0.192]
[0.192, 0.192, 0.192, 0.231, 0.192]
[0.192, 0.192, 0.231, 0.192, 0.192]
[0.192, 0.192, 0.192, 0.192, 0.231]

Caminho [0, 0, 2, 3, 4]
Rainhas
[0, 1, 0, 0, 0]
[1, 0, 0, 0, 0]
[0, 0, 0, 1, 0]
[0, 0, 1, 0, 0]
[0, 0, 0, 0, 1]

Feromonio
[1.2, 1.2, 1, 1, 1]
[1.4, 1, 1, 1, 1]
[1, 1, 1.2, 1.2, 1]
[1, 1, 1.2, 1.2, 1]
[1, 1, 1, 1, 1.4]

Probabilidade
[0.214, 0.231, 0.185, 0.185, 0.185]
[0.25, 0.192, 0.185, 0.185, 0.185]
[0.179, 0.192, 0.222, 0.222, 0.185]
[0.179, 0.192, 0.222, 0.222, 0.185]
[0.179, 0.192, 0.185, 0.185, 0.259]

Caminho [0, 1, 0, 3, 0]
Rainhas
[0, 1, 0, 0, 0]
[1, 0, 0, 0, 0]
[0, 0, 0, 1, 0]
[0, 0, 1, 0, 0]
[0, 0, 0, 0, 1]

Feromonio
[1.4, 1.2, 1, 1, 1]
[1.4, 1.2, 1, 1, 1]
[1.2, 1, 1.2, 

IndexError: list index out of range

In [46]:
sorteio = rdn.random()
pos0 = [0.192, 0.231, 0.192, 0.192, 0.192]
total = sum(pos0)
prob = [x*100/total for x in pos0]
prob

roleta = [(0, 1), (2, 3)]

for minimo, maximo in roleta:
    print(minimo, maximo)

0 1
2 3
